# Rudder equations
Proposed modification of the rudder drag equation. In the wPCC project it has been found that the rudder drag equation does not work very well in the situation where there is both drift angle and rudder angle. (This has been realized by comparing with VCT CFD calculations).

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sympy as sp
from sympy.plotting import plot as plot
from sympy.plotting import plot3d as plot3d
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
sp.init_printing()
from IPython.core.display import HTML,Latex

In [ ]:
import seaman.helpers
import seaman_symbol as ss
from rudder_equations import *
from bis_system import BisSystem

import seaman_symbols as symbols
from substitute_dynamic_symbols import lambdify, run

## Coordinate system
![coordinate_system](coordinateSystem.png)

## Effective rudder angle
The effective rudder angle, where the rudder "sees" both the oblique flow from rudder angle and the ship drift angle is the central part of the proposed change:

In [ ]:
effective_rudder_angle_equation

## Proposed change
The proposed change is that effective rudder angle $\delta_e$ should only be used for the calculation of transverse force $Y$ NOT $X$.

The rudder resistance is taken to be proportional to the rudder side force (without stall) and the
rudder angle, thus:

In [ ]:
rudder_drag_equation

The proposed change is that this $Y_{rudder}$ should not be calculated using $\delta_e$ when used for the calculation of $X_{rudder}$. If this is done, the proposed total rudder drag equation can then be written as:

In [ ]:
sp.latex(rudder_drag_equation_expanded)

In [ ]:
rudder_drag_equation_expanded_SI

This expression can be compared with the old one which looks like:

In [ ]:
sp.latex(rudder_drag_equation_expanded_old_SI)

## Validation of proposed change
The impact of the proposed change is validated using the VCT results from the wPCC project

In [ ]:
solution = sp.solve(rudder_drag_equation_expanded_old_SI, symbols.X_rudder, dict=True)[0][symbols.X_rudder]

In [ ]:
rudder_drag_function = sp.lambdify((delta, u_w, v_w, r_w, s, T_prop, n_prop, Y_Tdelta, Y_uudelta, X_Yrdelta, volume, rho, L,
                                    g, xx_rud,l_cg),
                                   sp.solve(rudder_drag_equation_expanded_SI,X_rudder, simplify=False)[0],
                                   modules='numpy',
                                   )
rudder_drag_function                                                             

In [ ]:
rudder_drag_function_old = sp.lambdify((delta, u_w, v_w, r_w, s, k_v, k_r, T_prop, n_prop, Y_Tdelta, Y_uudelta, X_Yrdelta, volume, rho, L,
                                    g, xx_rud,l_cg),
                                   sp.solve(rudder_drag_equation_expanded_old_SI,X_rudder, simplify=False)[0],
                                   modules='numpy',
                                   )
rudder_drag_function_old 

In [ ]:
import generate_input
ship_file_path='V3_0_R10.ship'
shipdict = seaman.ShipDict.load(ship_file_path)


### VCT results

In [ ]:
df_VCT = pd.read_csv('wPCC_V3_0_VCT.csv', index_col=0)

df_VCT.sort_values(by=['V','beta','delta'])[['V','beta','delta','fx']]


In [ ]:
fig,ax=plt.subplots()
df_VCT.sort_values(by='delta').plot(x='delta', y='fx', style='.', ax=ax)
ax.set_title('Rudder and drift angle test')

In [ ]:

df_input = generate_input.add_shipdict_inputs(lambda_function=rudder_drag_function,
                                              shipdict = shipdict,
                                              df = df_VCT,)
df_input

In [ ]:
result = df_input.copy()
result['fx'] = rudder_drag_function(**df_input)

df_input_old = generate_input.add_shipdict_inputs(lambda_function=rudder_drag_function_old,
                                              shipdict = shipdict,
                                              df = df_VCT,)
result_old = df_input_old.copy()
result_old['fx'] = rudder_drag_function_old(**df_input_old)

fig,ax=plt.subplots()
df_VCT.sort_values(by='delta').plot(x='delta', y='fx', style='.', label='VCT', ax=ax)
result.sort_values(by='delta').plot(x='delta', y='fx', style='o-', label='Proposal', ax=ax)
result_old.sort_values(by='delta').plot(x='delta', y='fx', style='^:', label='Old', ax=ax)
ax.set_xlabel(r'Rudder angle $\delta$ $[rad]$')
ax.set_ylabel(r'$F_x$ $[N]$')


ax.grid(True)
ax.set_title('Comparison of rudder drag')

# Conclusion
It was discovered from the VCT results in the wPCC project that the rudder drag was symetric with rudder angle $\delta$. The old seaman equation uses the effective rudder angle $\delta_e$ to obtain both $F_y$ and $F_x$. Using it only for $F_y$ is now proposed to get a symetric rudder drag, which seems to reproduce the CFD results better.